In [ ]:
import dask.dataframe as dask_pd
import dask_geopandas as dask_gpd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
import shapely
from dask.distributed import Client

In [ ]:
client = Client(n_workers=1, threads_per_worker=2, processes=True, memory_target_fraction=0.2, memory_limit='4GB')

In [ ]:
# dask_df_mdt =  dask_pd.read_csv('Compile-MDT/mdt*.csv')
dask_df_mdt =  dask_pd.read_csv('Compile-MDT/mdt*.csv', usecols=[0,1,2,3,4,6,7,8,9], assume_missing=True)
dask_df_mdt['rsrp'] = 'rsrp'

# grid = dask_pd.read_parquet('Grid Folder/grid_jabo100x100.parquet')
grid = dask_pd.read_parquet('Grid Folder/100x100_gridjabo.parquet')

In [ ]:
# grid = grid.drop(columns=['left', 'top', 'right', 'bottom'])
grid['geometry_polygon'] = grid['geometry'].astype(str)

In [ ]:
dask_df_mdt['pointer'] = "POINT (" + dask_df_mdt['longitude'].astype(str) + " " + dask_df_mdt['latitude'].astype(str) + ")"

In [ ]:
dask_gdf_mdt = dask_gpd.from_dask_dataframe(dask_df_mdt, geometry=dask_df_mdt["pointer"].map_partitions(
        gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)

# dask_gdf_mdt = dask_gdf_mdt.set_crs(4326)

In [ ]:
dask_gdf_grid = dask_gpd.from_dask_dataframe(grid, geometry=grid['geometry_polygon'].map_partitions(
        gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)

# dask_gdf_grid = dask_gdf_grid.set_crs(4326)

In [ ]:
dask_gdf_mdt = dask_gdf_mdt.sjoin(dask_gdf_grid, how='inner', predicate='within')

In [ ]:
dask_gdf_mdt = dask_gdf_mdt.drop(columns=['index_right', 'id', 'pointer'])

In [ ]:
# dask_df_mdt = dask_gdf_mdt.to_dask_dataframe()
dask_gdf_mdt['geometry_polygon'] = dask_gdf_mdt['geometry_polygon'].astype('str')

In [ ]:
dask_gdf_mdt['rsrp'] = dask_gdf_mdt['rsrp'].astype('category')
dask_gdf_mdt['rsrp'] = dask_gdf_mdt['rsrp'].cat.as_known()
pivot = dask_gdf_mdt.pivot_table(index='geometry_polygon', columns='rsrp', values='rsrp_serving', aggfunc='mean')

In [ ]:
pivot.to_csv('test.csv')